0. This notebook is the continuation to the preprocessing.ipynb notebook.

1. Import all neccessary modules 

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

2. Import X and y

In [2]:
import torchvision
from torchvision import transforms
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image

In [3]:
y = pd.read_csv('/mnt/c/Users/janos/OneDrive/Desktop/Master_thesis/Arousal.csv')

In [4]:
y = y.iloc[:,[1]]

In [5]:
y

,0
0,5.0
1,6.0
2,7.0
3,8.0
4,7.0
...,...
14271,5.0
14272,8.0
14273,2.0
14274,6.0


In [6]:
y_100_rows = y[0:100]

In [7]:
y_arr = y_100_rows.to_numpy()

In [8]:
os.chdir('/mnt/c/Users/janos/OneDrive/Desktop/Master_thesis')

NEW APPROACH:

Fit CNN on X

1. Read `n-rows`from X.

In [9]:
chunk = 10
i=0
X=[]
import pandas as pd
for df in pd.read_csv('X_matrix.csv',chunksize = chunk):
    X.append(df)
    i=i+1
    print('Processed {i}/10 chunks'.format(i=i))
    if i == 10:
        break



Processed 1/10 chunks
Processed 2/10 chunks
Processed 3/10 chunks
Processed 4/10 chunks
Processed 5/10 chunks
Processed 6/10 chunks
Processed 7/10 chunks
Processed 8/10 chunks
Processed 9/10 chunks
Processed 10/10 chunks


In [10]:
X = pd.concat(X)

In [11]:
del X [X.columns[0]]

In [12]:
X_arr = X.to_numpy()

In [77]:
X_arr = np.reshape(X_arr,(100,224,224))

In [78]:
X_arr.shape

(100, 224, 224)

Adding new dimension for CNN

In [79]:
X_ = X_arr[:,:,np.newaxis]

In [80]:
X_ = np.swapaxes(X_,2,1)

In [81]:
X_.shape

(100, 1, 224, 224)

Converting X to appropriate Datatype...

In [82]:
X_tensor_train = torch.from_numpy(X_)
y_tensor_train = torch.from_numpy(y_arr)

In [83]:
X_tensor_train = torch.tensor(X_tensor_train,dtype = torch.float32)
y_tensor_train = y_tensor_train.type(torch.LongTensor)

/tmp/ipykernel_1215/1671349079.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_tensor_train = torch.tensor(X_tensor_train,dtype = torch.float32)


Pytorch needs the true labels to be in the range N-1. Thus, substract 1 one from each label.

In [84]:
y_tensor_train = y_tensor_train-1

In [85]:
X_tensor_train.shape

torch.Size([100, 1, 224, 224])

In [86]:
y_tensor_train.shape

torch.Size([100, 1])

Setup custom dataset class

In [87]:
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        X = self.X[idx]
        y = self.y[idx]
        return X, y


Setting up the CNN.

In [120]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.fc1 = nn.Linear(774400, 128)
        self.fc2 = nn.Linear(128, 10)
        self.pool = nn.MaxPool2d(2)
        
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = torch.flatten(x, 1)
        print(x.shape)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x


In [121]:
SalienceNet = CNN()

In [108]:
dataset = CustomDataset(X_tensor_train,y_tensor_train)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
criterion = nn.CrossEntropyLoss()


In [119]:
for x,y in dataloader:
    print(y)

tensor([[2],
        [1],
        [3],
        [6],
        [6],
        [7],
        [4],
        [7],
        [5],
        [5],
        [4],
        [6],
        [6],
        [5],
        [7],
        [5],
        [5],
        [4],
        [4],
        [1],
        [6],
        [5],
        [5],
        [6],
        [5],
        [7],
        [4],
        [5],
        [6],
        [7],
        [5],
        [6]])
tensor([[5],
        [6],
        [5],
        [5],
        [4],
        [5],
        [3],
        [6],
        [5],
        [3],
        [7],
        [5],
        [3],
        [6],
        [5],
        [3],
        [4],
        [5],
        [4],
        [4],
        [5],
        [6],
        [4],
        [4],
        [4],
        [6],
        [4],
        [5],
        [4],
        [5],
        [6],
        [7]])
tensor([[6],
        [6],
        [4],
        [6],
        [5],
        [6],
        [6],
        [6],
        [5],
        [4],
        [4],
        [7],
        [4

In [116]:
def train(net, loss_fn, dataloader,
          epoch=100, learning_rate=0.000000001):
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)#,weight_decay=1e-4)
    train_loss = []
    
    for i in range(epoch):   
        net.train()
        for data, label in dataloader:
            data, label = data.to('cpu'), label.to('cpu')
        #make prediction
            output = net(data)

            # Zero out the gradients
            optimizer.zero_grad()

            # Calculate loss.
            loss = criterion(output,torch.max(label, 1)[0])

            # Backpropagation and gradient update.
            loss.backward()# Calculate gradients. #retrain_graph=True

            optimizer.step() # Apply gradient udpate.

            train_loss.append(loss.item())
            
            if (i + 1) % (epoch // 5) == 0:
                print(f'iteration {i + 1}/{epoch} | loss: {loss.item():.3f}')
    y_pred = torch.softmax(output,dim=1).argmax(dim=1)
    return y_pred, train_loss, output,i


In [26]:
import matplotlib.pyplot as plt

In [ ]:
np.random.seed(1)
torch.manual_seed(1)

l,p,o,index=train(net=SalienceNet,loss_fn=criterion,dataloader=dataloader,epoch=10)

plt.plot(p)

plt.xlabel('Iterations of gradient descent')
plt.ylabel('Cross entropy')
plt.show()

torch.Size([32, 774400])
torch.Size([32, 774400])
torch.Size([32, 774400])
torch.Size([4, 774400])
torch.Size([32, 774400])
iteration 2/10 | loss: 2.284
torch.Size([32, 774400])
iteration 2/10 | loss: 2.302
torch.Size([32, 774400])
iteration 2/10 | loss: 2.304
torch.Size([4, 774400])
iteration 2/10 | loss: 2.284
torch.Size([32, 774400])
